<a href="https://colab.research.google.com/github/S1u2m3e4e5t6/LexiVoice-AI/blob/main/Lexi_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.8/241.8 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.4/224.4 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.6/355.6 kB 32.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.9.0
    Uninstalling fsspec-2025.9.0:
      Successfully uninstalled fsspec-2025.9.0


In [3]:
pip install -U datasets

In [5]:
from datasets import load_dataset

ds = load_dataset("ShreyasP123/Legal-Dataset-for-india")

README.md: 0.00B [00:00, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


final_chunk.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1469 [00:00<?, ? examples/s]

In [ ]:
!pip install tiktoken
!pip install datasets
!pip install numpy
!pip install tqdm

# Import libraries
import tiktoken
import os
import numpy as np
from tqdm.auto import tqdm
from datasets import load_dataset # 'datasets' library to load the dataset

# Get the tokenizer encoding for gpt2
enc = tiktoken.get_encoding("gpt2")

# Define a function to process each example in the dataset
def process(example):
    # encode_ordinary ignores any special tokens
    ids = enc.encode_ordinary(example['text'])
    out = {'ids': ids, 'len': len(ids)}
    return out

# --- Fix starts here ---
# Load a dataset. You can replace 'wikitext' with your desired dataset.
# The variable 'ds' is now defined here.
ds = load_dataset('wikitext', 'wikitext-103-raw-v1')
# --- Fix ends here ---

# Check if the training file already exists to avoid reprocessing
if not os.path.exists("train.bin"):
    # Tokenize the dataset using the 'process' function
    tokenized = ds.map(
        process,
        remove_columns=['text'],
        desc="tokenizing the splits",
        num_proc=8,
    )
    # Concatenate all the token IDs in each dataset split into a single large file for training
    for split, dset in tokenized.items():
        arr_len = np.sum(dset['len'], dtype=np.uint64)
        filename = f'{split}.bin'
        # The dtype can be uint16 since enc.max_token_value (50256) is < 2**16
        dtype = np.uint16
        arr = np.memmap(filename, dtype=dtype, mode='w+', shape=(arr_len,))
        total_batches = 1024

        idx = 0
        for batch_idx in tqdm(range(total_batches), desc=f'writing {filename}'):
            # Batch together samples for faster writing
            batch = dset.shard(num_shards=total_batches, index=batch_idx, contiguous=True).with_format('numpy')
            arr_batch = np.concatenate(batch['ids'])
            # Write the batch to the memory-mapped file
            arr[idx : idx + len(arr_batch)] = arr_batch
            idx += len(arr_batch)
        arr.flush()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.0 MB/s eta 0:00:00


README.md: 0.00B [00:00, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/157M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/157M [00:00<?, ?B/s]

...

tokenizing the splits (num_proc=8):   0%|          | 0/4358 [00:00<?, ? examples/s]

tokenizing the splits (num_proc=8):   0%|          | 0/1801350 [00:00<?, ? examples/s]

tokenizing the splits (num_proc=8):   0%|          | 0/3760 [00:00<?, ? examples/s]

writing test.bin:   0%|          | 0/1024 [00:00<?, ?it/s]

writing train.bin:   0%|          | 0/1024 [00:00<?, ?it/s]